# HW05 — Бейзлайн vs Logistic Regression (Pipeline)

**Цель:** сравнить простой бейзлайн (`DummyClassifier`) и логистическую регрессию (`LogisticRegression`) на датасете дефолтов.

> Важно: ноутбук предполагается запускать из папки `homeworks/HW05/`, поэтому пути к файлам — **относительные**.


## 2.3.1 Импорты, загрузка датасета и первичный анализ (EDA-lite)

In [ ]:
# Базовые библиотеки (как в demo)
import numpy as np
import pandas as pd

# Воспроизводимость
RANDOM_STATE = 42

# sklearn: разбиение и модели
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# метрики
from sklearn.metrics import (
    accuracy_score, roc_auc_score,
    precision_score, recall_score, f1_score,
    roc_curve, precision_recall_curve, average_precision_score
)

import matplotlib.pyplot as plt


In [ ]:
# Загрузка датасета
df = pd.read_csv("S05-hw-dataset.csv")
print("Shape:", df.shape)
df.head()

In [ ]:
# Информация о столбцах и типах
df.info()


In [ ]:
# Описательные статистики
df.describe()

In [ ]:
# Распределение таргета
print(df["default"].value_counts())
print()
print(df["default"].value_counts(normalize=True))


**Короткие наблюдения:**

- В датасете 3000 объектов и 17 столбцов (из них default — таргет, client_id — технический идентификатор; для модели его обычно исключают).

- Пропусков не обнаружено (можно подтвердить df.isna().sum()), типы — в основном целочисленные, несколько вещественных.

- По диапазонам признаков (возраст, доход, кредитный скор, debt_to_income, бинарные флаги) явных выбросов по условиям из описания нет.

- Есть аномалия по смыслу: у части клиентов years_employed > age (276 строк, ~9.2%), что невозможно, если years_employed — “стаж работы в годах”. Это стоит отметить как качество данных/синтетическую особенность.

- Таргет умеренно несбалансирован: default=0 — 1769 (≈58.97%), default=1 — 1231 (≈41.03%).


## 2.3.2 Подготовка признаков и таргета

In [ ]:
# X — все признаки кроме default и client_id, y — таргет default
y = df["default"].astype(int)
X = df.drop(columns=["default", "client_id"])

print("X shape:", X.shape)
print("y shape:", y.shape)
print("All numeric:", X.select_dtypes(exclude=[np.number]).shape[1] == 0)

# простая проверка диапазонов (по желанию)
print("debt_to_income out of [0,1]:", ((X["debt_to_income"] < 0) | (X["debt_to_income"] > 1)).sum())


## 2.3.3 Train/Test-сплит и бейзлайн-модель

In [ ]:
# Train/Test split (как в demo: фиксируем random_state и используем stratify)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)
print("Target share train:", y_train.value_counts(normalize=True).round(4).to_dict())
print("Target share test :", y_test.value_counts(normalize=True).round(4).to_dict())


In [ ]:
# Бейзлайн: DummyClassifier (most_frequent)
baseline = DummyClassifier(strategy="most_frequent", random_state=RANDOM_STATE)
baseline.fit(X_train, y_train)

y_pred_dummy = baseline.predict(X_test)
y_proba_dummy = baseline.predict_proba(X_test)[:, 1]

base_acc = accuracy_score(y_test, y_pred_dummy)
base_auc = roc_auc_score(y_test, y_proba_dummy)

print("=== DummyClassifier (most_frequent) – test ===")
print("Accuracy:", base_acc)
print("ROC-AUC :", base_auc)


**Объяснение:**

- most_frequent всегда предсказывает самый частый класс (здесь это default=0), поэтому accuracy примерно равен доле нулевого класса, а ROC-AUC ≈ 0.5 (модель не различает классы).

- stratified предсказывает классы случайно, сохраняя пропорции классов из train, поэтому и accuracy, и ROC-AUC близки к случайному угадыванию.

- Бейзлайн важен как точка отсчёта: любая нормальная модель должна уверенно превосходить эти значения, иначе она не приносит пользы.

## 2.3.4 Логистическая регрессия (Pipeline) и подбор гиперпараметра `C`

In [ ]:
# Pipeline: StandardScaler + LogisticRegression
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=2000, solver="lbfgs"))
])

In [ ]:
# Подбор C простым перебором — быстро и воспроизводимо
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_train
)

C_list = [0.01, 0.1, 1.0, 10.0]
best_auc_val, best_C = -1.0, None

for C in C_list:
    pipe.set_params(logreg__C=C)
    pipe.fit(X_tr, y_tr)
    y_val_proba = pipe.predict_proba(X_val)[:, 1]
    auc_val = roc_auc_score(y_val, y_val_proba)
    print(f"C={C:<5} | val ROC-AUC={auc_val:.4f}")
    if auc_val > best_auc_val:
        best_auc_val, best_C = auc_val, C

print("\nBest C:", best_C, "| Best val ROC-AUC:", round(best_auc_val, 4))


In [ ]:
# Обучаем лучшую модель на полном train и оцениваем на test
pipe.set_params(logreg__C=best_C)
pipe.fit(X_train, y_train)

y_pred_lr = pipe.predict(X_test)
y_proba_lr = pipe.predict_proba(X_test)[:, 1]

lr_acc = accuracy_score(y_test, y_pred_lr)
lr_auc = roc_auc_score(y_test, y_proba_lr)

print("=== LogisticRegression (best C) – test ===")
print("Accuracy:", lr_acc)
print("ROC-AUC :", lr_auc)

# дополнительные метрики (опционально, но полезно)
print("Precision:", precision_score(y_test, y_pred_lr))
print("Recall   :", recall_score(y_test, y_pred_lr))
print("F1-score :", f1_score(y_test, y_pred_lr))


In [ ]:
# ROC curve + сохранение в figures/
fpr, tpr, _ = roc_curve(y_test, y_proba_lr)

plt.figure()
plt.plot(fpr, tpr, label=f"LogReg (AUC={lr_auc:.3f})")
plt.plot([0, 1], [0, 1], linestyle="--", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve (TEST)")
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

plt.savefig("figures/roc_curve.png", dpi=500)
plt.show()

## 2.3.5 Сравнение моделей и выводы

In [ ]:
# Табличка сравнения (Dummy vs LogReg)
summary = pd.DataFrame([
    {"model": "Dummy (most_frequent)", "accuracy": base_acc, "roc_auc": base_auc},
    {"model": f"LogReg (best C={best_C})", "accuracy": lr_acc, "roc_auc": lr_auc},
])

summary

In [ ]:
# Насколько выросли метрики (LogReg - Dummy)
delta = summary.loc[1, ["accuracy", "roc_auc"]] - summary.loc[0, ["accuracy", "roc_auc"]]
print("Δaccuracy:", float(delta["accuracy"]))
print("ΔROC-AUC :", float(delta["roc_auc"]))


In [ ]:
report = f"""В качестве точки отсчёта использовалась бейзлайн-модель DummyClassifier (most_frequent), которая всегда предсказывает самый частый класс.
Её качество на тестовой выборке: accuracy = {base_acc:.3f}, ROC-AUC = {base_auc:.3f}.
Логистическая регрессия (Pipeline: StandardScaler + LogisticRegression) показала заметно лучшее качество: accuracy = {lr_acc:.3f}, ROC-AUC = {lr_auc:.3f}.
Прирост по сравнению с бейзлайном составил: Δaccuracy = {lr_acc - base_acc:+.3f}, ΔROC-AUC = {lr_auc - base_auc:+.3f}.
При переборе нескольких значений C качество по ROC-AUC менялось незначительно (на валидации), поэтому модель стабильна к разумным изменениям силы регуляризации.
Лучшая настройка на нашей проверке: C = {best_C}.
В целом логистическая регрессия выглядит разумной для этой задачи: она проста, интерпретируема и существенно превосходит бейзлайн по ROC-AUC.
"""
print(report)


## 2.4 Опционально: PR-кривая и метрики при пороге

In [ ]:
# PR curve + Average Precision (AP) + сохранение
precision, recall, _ = precision_recall_curve(y_test, y_proba_lr)
ap = average_precision_score(y_test, y_proba_lr)

plt.figure()
plt.plot(recall, precision, label=f"LogReg (AP={ap:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall curve (TEST)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.savefig("figures/pr_curve.png", dpi=500)
plt.show()

print("Average Precision:", ap)


In [ ]:
# precision/recall/f1 для выбранного порога (например, 0.5)
threshold = 0.5
y_pred_thr = (y_proba_lr >= threshold).astype(int)

print("Threshold:", threshold)
print("precision:", precision_score(y_test, y_pred_thr))
print("recall   :", recall_score(y_test, y_pred_thr))
print("f1       :", f1_score(y_test, y_pred_thr))
